In [1]:
import os
os.chdir("../")
%pwd

'd:\\Machine_Learning\\Consignment-Pricing-Prediction'

In [257]:
import pandas as pd

df = pd.read_csv("artifacts\data_ingestion\SCMS_Delivery_History_Dataset.csv", usecols=[
    'Project Code', 'PQ #', 'Country', 'Fulfill Via', 'Vendor INCO Term',
       'Shipment Mode', 'PQ First Sent to Client Date',
       'Delivery Recorded Date', 'Product Group', 'Sub Classification',
       'Vendor', 'Item Description', 'Molecule/Test Type', 'Brand', 'Dosage',
       'Dosage Form', 'Unit of Measure (Per Pack)', 'Line Item Quantity',
       'Line Item Value', 'Pack Price', 'Unit Price', 'Manufacturing Site',
       'Weight (Kilograms)', 'Freight Cost (USD)', 'Line Item Insurance (USD)','Scheduled Delivery Date', 'Delivered to Client Date'])

# df = pd.read_csv('artifacts/data_ingestion/SCMS_Delivery_History_Dataset.csv', index_col=['ID', 'PO / SO #', 'ASN/DN #', 'Managed By', 'PO Sent to Vendor Date',
#        'First Line Designation'])

In [258]:
df.columns 

Index(['Project Code', 'PQ #', 'Country', 'Fulfill Via', 'Vendor INCO Term',
       'Shipment Mode', 'PQ First Sent to Client Date',
       'Scheduled Delivery Date', 'Delivered to Client Date',
       'Delivery Recorded Date', 'Product Group', 'Sub Classification',
       'Vendor', 'Item Description', 'Molecule/Test Type', 'Brand', 'Dosage',
       'Dosage Form', 'Unit of Measure (Per Pack)', 'Line Item Quantity',
       'Line Item Value', 'Pack Price', 'Unit Price', 'Manufacturing Site',
       'Weight (Kilograms)', 'Freight Cost (USD)',
       'Line Item Insurance (USD)'],
      dtype='object')

In [259]:
df.head()

,Project Code,PQ #,Country,Fulfill Via,Vendor INCO Term,Shipment Mode,PQ First Sent to Client Date,Scheduled Delivery Date,Delivered to Client Date,Delivery Recorded Date,...,Dosage Form,Unit of Measure (Per Pack),Line Item Quantity,Line Item Value,Pack Price,Unit Price,Manufacturing Site,Weight (Kilograms),Freight Cost (USD),Line Item Insurance (USD)
0,100-CI-T01,Pre-PQ Process,Côte d'Ivoire,Direct Drop,EXW,Air,Pre-PQ Process,2-Jun-06,2-Jun-06,2-Jun-06,...,Test kit,30,19,551.0,29.00,0.97,Ranbaxy Fine Chemicals LTD,13,780.34,NaN
1,108-VN-T01,Pre-PQ Process,Vietnam,Direct Drop,EXW,Air,Pre-PQ Process,14-Nov-06,14-Nov-06,14-Nov-06,...,Oral suspension,240,1000,6200.0,6.20,0.03,"Aurobindo Unit III, India",358,4521.5,NaN
2,100-CI-T01,Pre-PQ Process,Côte d'Ivoire,Direct Drop,FCA,Air,Pre-PQ Process,27-Aug-06,27-Aug-06,27-Aug-06,...,Test kit,100,500,40000.0,80.00,0.80,ABBVIE GmbH & Co.KG Wiesbaden,171,1653.78,NaN
3,108-VN-T01,Pre-PQ Process,Vietnam,Direct Drop,EXW,Air,Pre-PQ Process,1-Sep-06,1-Sep-06,1-Sep-06,...,Tablet,60,31920,127360.8,3.99,0.07,"Ranbaxy, Paonta Shahib, India",1855,16007.06,NaN
4,108-VN-T01,Pre-PQ Process,Vietnam,Direct Drop,EXW,Air,Pre-PQ Process,11-Aug-06,11-Aug-06,11-Aug-06,...,Capsule,60,38000,121600.0,3.20,0.05,"Aurobindo Unit III, India",7590,45450.08,NaN


In [260]:
df.shape

(10324, 27)

In [261]:
# df =df.dropna()

In [262]:
type(df)

pandas.core.frame.DataFrame

In [263]:
from sklearn.pipeline import FunctionTransformer, make_pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

def transform_dataset(testDF: pd.DataFrame):
    # Droping the na values from the dataframe
    testDF=testDF.dropna()

    # Checking the object values in the Weight Column
    to_drop_weight= []
    for i in testDF['Weight (Kilograms)']: 
        try:
            float(i)
        except:
            to_drop_weight.append(i)
    # Dropping the object values from the Weight Column
    testDF = testDF[testDF['Weight (Kilograms)'].isin(to_drop_weight)==False]

    # Checking the object values in the Freight Column
    to_drop= []
    for i in testDF['Freight Cost (USD)']:
        try:
            float(i)
        except:
            to_drop.append(i)        
    # Dropping the object values from the Freight Column
    testDF = testDF[testDF['Freight Cost (USD)'].isin(to_drop)==False]

    # Splitting the scheduled Delivery date column to three respective columns storing the date, month and year
    testDF['Scheduled_Delivery_Date_D'] = [i.split('-')[0] for i in testDF['Scheduled Delivery Date']]
    testDF['Scheduled_Delivery_Date_M'] = [i.split('-')[1] for i in testDF['Scheduled Delivery Date']]
    testDF['Scheduled_Delivery_Date_Y'] = [i.split('-')[2] for i in testDF['Scheduled Delivery Date']]

    # Splitting the delivered to client date column to three respective columns storing the date, month and year
    testDF['Delivered_to_Client_Date_D'] = [i.split('-')[0] for i in testDF['Delivered to Client Date']]
    testDF['Delivered_to_Client_Date_M'] = [i.split('-')[1] for i in testDF['Delivered to Client Date']]
    testDF['Delivered_to_Client_Date_Y'] = [i.split('-')[2] for i in testDF['Delivered to Client Date']]

    # Dropping the scheduled delivery date column and the client date column
    testDF = testDF.drop(columns=['Delivered to Client Date', 'Scheduled Delivery Date'], axis= 1)

    # Changing the datatype of the columns
    testDF['Weight (Kilograms)'] = testDF['Weight (Kilograms)'].astype(float)
    testDF['Freight Cost (USD)'] = testDF['Freight Cost (USD)'].astype(float)
    testDF['Scheduled_Delivery_Date_D'] = testDF['Scheduled_Delivery_Date_D'].astype(float)
    testDF['Scheduled_Delivery_Date_Y'] = testDF['Scheduled_Delivery_Date_Y'].astype(float)
    testDF['Delivered_to_Client_Date_D'] = testDF['Delivered_to_Client_Date_D'].astype(float)
    testDF['Delivered_to_Client_Date_Y'] = testDF['Delivered_to_Client_Date_Y'].astype(float)

    # Encoding the data
    # le = LabelEncoder()
    # testDF = testDF.apply(le.fit_transform)

    # # Scaling the data
    # scaler = StandardScaler()
    # testDF = scaler.fit_transform(testDF)
    
    return testDF

In [264]:
df = transform_dataset(df)

In [265]:
df

,Project Code,PQ #,Country,Fulfill Via,Vendor INCO Term,Shipment Mode,PQ First Sent to Client Date,Delivery Recorded Date,Product Group,Sub Classification,...,Manufacturing Site,Weight (Kilograms),Freight Cost (USD),Line Item Insurance (USD),Scheduled_Delivery_Date_D,Scheduled_Delivery_Date_M,Scheduled_Delivery_Date_Y,Delivered_to_Client_Date_D,Delivered_to_Client_Date_M,Delivered_to_Client_Date_Y
22,108-VN-T01,Pre-PQ Process,Vietnam,Direct Drop,EXW,Air,Pre-PQ Process,15-Oct-07,ARV,Pediatric,...,"Aurobindo Unit III, India",941.0,4193.49,3.07,15.0,Oct,7.0,15.0,Oct,7.0
23,108-VN-T01,Pre-PQ Process,Vietnam,Direct Drop,FCA,Air,Pre-PQ Process,27-Aug-07,ARV,Adult,...,ABBVIE Ludwigshafen Germany,117.0,1767.38,65.75,27.0,Aug,7.0,27.0,Aug,7.0
29,117-ET-T01,Pre-PQ Process,Ethiopia,Direct Drop,EXW,Air,Pre-PQ Process,20-Nov-07,ARV,Adult,...,"Aurobindo Unit III, India",4228.0,12237.61,158.72,20.0,Nov,7.0,20.0,Nov,7.0
32,108-VN-T01,Pre-PQ Process,Vietnam,Direct Drop,EXW,Air,Pre-PQ Process,21-Jan-08,ARV,Adult,...,Gilead(Nycomed) Oranienburg DE,76.0,2282.57,27.20,21.0,Jan,8.0,21.0,Jan,8.0
33,108-VN-T01,Pre-PQ Process,Vietnam,Direct Drop,EXW,Air,Pre-PQ Process,21-Jan-08,ARV,Adult,...,"Ranbaxy, Paonta Shahib, India",23.0,2149.05,5.36,21.0,Jan,8.0,21.0,Jan,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10305,111-MZ-T30,FPQ-15195,Mozambique,From RDC,N/A - From RDC,Truck,10/17/14,15-Jun-15,ARV,Adult,...,Hetero Unit III Hyderabad IN,39688.0,28277.42,2725.93,29.0,May,15.0,9.0,Jun,15.0
10306,111-MZ-T30,FPQ-14843,Mozambique,From RDC,N/A - From RDC,Truck,8/21/14,15-Jun-15,ARV,Adult,...,"Hetero, Jadcherla, unit 5, IN",1504.0,1653.90,75.52,29.0,May,15.0,15.0,Jun,15.0
10310,114-UG-T30,FPQ-15294,Uganda,From RDC,N/A - From RDC,Truck,11/3/14,15-Jun-15,ARV,Pediatric,...,Mylan (formerly Matrix) Nashik,2253.0,3785.72,76.27,30.0,Jun,15.0,4.0,Jun,15.0
10315,151-NG-T30,FPQ-14989,Nigeria,From RDC,N/A - From RDC,Air Charter,9/19/14,22-May-15,ARV,Adult,...,ABBVIE Ludwigshafen Germany,15198.0,26180.00,1341.33,30.0,Jun,15.0,15.0,May,15.0


In [266]:
cat_col = df.select_dtypes(object).columns

In [267]:
cat_col 

Index(['Project Code', 'PQ #', 'Country', 'Fulfill Via', 'Vendor INCO Term',
       'Shipment Mode', 'PQ First Sent to Client Date',
       'Delivery Recorded Date', 'Product Group', 'Sub Classification',
       'Vendor', 'Item Description', 'Molecule/Test Type', 'Brand', 'Dosage',
       'Dosage Form', 'Manufacturing Site', 'Scheduled_Delivery_Date_M',
       'Delivered_to_Client_Date_M'],
      dtype='object')

In [268]:
df[cat_col].nunique()

Project Code                      81
PQ #                             815
Country                           31
Fulfill Via                        2
Vendor INCO Term                   7
Shipment Mode                      4
PQ First Sent to Client Date     547
Delivery Recorded Date          1582
Product Group                      3
Sub Classification                 4
Vendor                            34
Item Description                 111
Molecule/Test Type                40
Brand                             21
Dosage                            49
Dosage Form                       15
Manufacturing Site                50
Scheduled_Delivery_Date_M         12
Delivered_to_Client_Date_M        12
dtype: int64

In [269]:
df['Project Code'].unique()

array(['108-VN-T01', '117-ET-T01', '109-TZ-T01', '112-NG-T01',
       '104-CI-T01', '123-NG-T01', '106-HT-T01', '107-RW-T01',
       '131-NG-T01', '114-UG-T01', '119-NA-T01', '105-GY-T01',
       '130-NG-T01', '100-ZW-T01', '111-MZ-T01', '110-ZM-T01',
       '126-NG-T01', '151-NG-T01', '100-SZ-T01', '132-NG-T01',
       '113-ZW-T01', '153-NG-T01', '151-NG-T30', '108-VN-T30',
       '106-HT-T30', '111-MZ-T30', '161-ZA-T30', '107-RW-T30',
       '103-MW-T30', '104-CI-T30', '114-UG-T30', '113-ZW-T30',
       '101-CD-T30', '109-TZ-T30', '110-ZM-T30', '100-BJ-T30',
       '117-ET-T30', '900-TZ-T30', '112-NG-T30', '116-ZA-T30',
       '105-GY-T30', '901-CM-T30', '123-NG-T30', '122-HT-T30',
       '901-NA-T30', '103-ZW-T30', '105-GH-T30', '119-NA-T30',
       '200-ZW-T30', '102-ZW-T30', '900-TG-T30', '901-GH-T30',
       '103-AO-T30', '103-CM-T30', '101-AF-T30', '200-DO-T30',
       '900-CM-T30', '202-GT-T30', '100-DO-T30', '103-UG-T30',
       '900-UG-T30', '103-DO-T30', '900-SZ-T30', '103-S

In [270]:
feature_map = {}

In [271]:
import numpy as np
for i in cat_col:
    feature_map[i] = df[i].unique()

In [272]:
feature_map

{'Project Code': array(['108-VN-T01', '117-ET-T01', '109-TZ-T01', '112-NG-T01',
        '104-CI-T01', '123-NG-T01', '106-HT-T01', '107-RW-T01',
        '131-NG-T01', '114-UG-T01', '119-NA-T01', '105-GY-T01',
        '130-NG-T01', '100-ZW-T01', '111-MZ-T01', '110-ZM-T01',
        '126-NG-T01', '151-NG-T01', '100-SZ-T01', '132-NG-T01',
        '113-ZW-T01', '153-NG-T01', '151-NG-T30', '108-VN-T30',
        '106-HT-T30', '111-MZ-T30', '161-ZA-T30', '107-RW-T30',
        '103-MW-T30', '104-CI-T30', '114-UG-T30', '113-ZW-T30',
        '101-CD-T30', '109-TZ-T30', '110-ZM-T30', '100-BJ-T30',
        '117-ET-T30', '900-TZ-T30', '112-NG-T30', '116-ZA-T30',
        '105-GY-T30', '901-CM-T30', '123-NG-T30', '122-HT-T30',
        '901-NA-T30', '103-ZW-T30', '105-GH-T30', '119-NA-T30',
        '200-ZW-T30', '102-ZW-T30', '900-TG-T30', '901-GH-T30',
        '103-AO-T30', '103-CM-T30', '101-AF-T30', '200-DO-T30',
        '900-CM-T30', '202-GT-T30', '100-DO-T30', '103-UG-T30',
        '900-UG-T30', '1

In [175]:
import pickle
obj = transform_dataset
with open('preprocessor.pkl', 'wb') as file:
            pickle.dump(obj, file)

In [176]:
import pickle 
loaded_preprocessor = None
with open('labelEncoder.pkl', 'rb') as file:
    loaded_preprocessor =pickle.load(file)
# df = transform_dataset(df)
# df['Project Code'] = loaded_preprocessor.transform(df['Project Code'])
# type(loaded_preprocessor)
df = df.apply(loaded_preprocessor.inverse_transform)

ValueError: y contains previously unseen labels: [ 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32
 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56
 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80]

In [109]:
test = {
    'df': [1,11,1,11,1,1,1,1,1],
    'df2': [2,22,2,22,2,2,np.NAN,2,2],
    'df3': [3,33,3,33,3,np.NAN,3,3,3],
    'df4': [4,44,4,np.NAN,4,4,4,4,4],
}
test_df = pd.DataFrame(test)
test_df
# test_df.dropna(inplace=True)

,df,df2,df3,df4
0,1,2.0,3.0,4.0
1,11,22.0,33.0,44.0
2,1,2.0,3.0,4.0
3,11,22.0,33.0,NaN
4,1,2.0,3.0,4.0
5,1,2.0,NaN,4.0
6,1,NaN,3.0,4.0
7,1,2.0,3.0,4.0
8,1,2.0,3.0,4.0
